In [23]:
# Import necessary modules
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os

# Import CRUD module
from animal_shelter import AnimalShelter

# Database credentials
username = "aacuser"
password = "NewSecurePassword"

# Connect to MongoDB
db = AnimalShelter(username, password)

# Retrieve all documents from MongoDB
df = pd.DataFrame.from_records(db.read({}))

# Remove MongoDB '_id' column to prevent errors in Dash table
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Define strict filtering criteria
rescue_criteria = {
    "Water Rescue": {
        "breeds": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": "Intact Female",
        "age_range": (26, 156)
    },
    "Mountain Rescue": {
        "breeds": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        "sex": "Intact Male",
        "age_range": (26, 156)
    },
    "Disaster/Tracking": {
        "breeds": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
        "sex": "Intact Male",
        "age_range": (20, 300)
    }
}

# Initialize Dash app
app = JupyterDash(__name__)

# Load Grazioso Salvare’s Logo
image_filename = "Grazioso Salvare Logo.png"
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
else:
    encoded_image = ""

# Define Layout
app.layout = html.Div([
    html.Center(html.Img(src=f"data:image/png;base64,{encoded_image}", style={'width': '200px'})),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H3("Unique Identifier: Victor Tran"))),  # Unique ID added
    html.Hr(),

    # Filter by Rescue Type
    html.Label("Filter by Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Reset', 'value': 'Reset'},  # Changed from "All" to "Reset"
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
            {'label': 'Disaster/Tracking', 'value': 'Disaster/Tracking'}
        ],
        value='Reset',  # Default selection is Reset
        labelStyle={'display': 'block'}
    ),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable='single',
        selected_rows=[0]
    ),

    html.Br(),
    html.Hr(),

    # Graph and Map Containers
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callback to filter data based on strict rescue type criteria
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == "Reset":  # Changed from "All" to "Reset"
        df_filtered = df  # No filtering, return full dataset
    else:
        criteria = rescue_criteria.get(filter_type, {})
        allowed_breeds = criteria.get("breeds", [])
        preferred_sex = criteria.get("sex", "")
        min_age, max_age = criteria.get("age_range", (0, 9999))

        # Apply strict filtering: Only exact breeds, correct sex, and valid age range
        df_filtered = df[
            df['breed'].isin(allowed_breeds) &
            (df['sex_upon_outcome'] == preferred_sex) &
            (df['age_upon_outcome_in_weeks'].between(min_age, max_age))
        ]

    return df_filtered.to_dict('records')

# Callback to update breed distribution chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', "value")]
)
def update_graphs(viewData, filter_type):
    if not viewData:
        return "No data available."
    
    dff = pd.DataFrame.from_dict(viewData)

    # Count breed occurrences
    breed_counts = dff['breed'].value_counts(normalize=True) * 100  # Convert to percentage

    # Apply grouping logic only when "Reset" is selected
    if filter_type == "Reset":
        threshold = 2  # Breeds with less than 2% are grouped
        large_breeds = breed_counts[breed_counts >= threshold]
        small_breeds = breed_counts[breed_counts < threshold].sum()

        # Create a new DataFrame for the pie chart
        breed_chart_data = large_breeds.reset_index()
        breed_chart_data.columns = ['Breed', 'Percentage']

        # Add "Other" category if needed
        if small_breeds > 0:
            breed_chart_data = breed_chart_data.append({'Breed': 'Other', 'Percentage': small_breeds}, ignore_index=True)
    else:
        # No grouping for filtered selections
        breed_chart_data = breed_counts.reset_index()
        breed_chart_data.columns = ['Breed', 'Percentage']

    # Generate pie chart
    return dcc.Graph(
        figure=px.pie(
            breed_chart_data,
            names='Breed',
            values='Percentage',
            title='Breed Distribution'
        )
    )

# Callback to highlight selected row
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update map based on selection
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or index is None or not index:
        return "No map data available."
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0  

    return dl.Map(style={'width': '1000px', 'height': '500px'},
                  center=[30.75, -97.48], zoom=10, children=[
                      dl.TileLayer(id="base-layer-id"),
                      dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                                children=[
                                    dl.Tooltip(dff.iloc[row, 4]),
                                    dl.Popup([
                                        html.H1("Animal Name"),
                                        html.P(dff.iloc[row, 9])
                                    ])
                                ])
                  ])

# Run app
app.run_server(debug=True)


MongoDB Connection Successful!
Dash app running on http://127.0.0.1:29710/


/tmp/ipykernel_6523/1269274585.py:151: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

